In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.applications import vgg16
from keras.applications import mobilenet
from tensorflow.keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
train_path = 'data/train'
valid_path = 'data/valid'
test_path = 'data/test'

In [5]:
train_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input).flow_from_directory(train_path, target_size=(224,224), batch_size=30)
valid_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input).flow_from_directory(valid_path, target_size=(224,224), batch_size=30)
test_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input).flow_from_directory(test_path, target_size=(224,224), batch_size=30)

Found 202 images belonging to 2 classes.
Found 103 images belonging to 2 classes.
Found 451 images belonging to 2 classes.


In [6]:
base_model = vgg16.VGG16(weights="imagenet", include_top=False, input_shape=(224,224,3))
base_model.summary()

58900480/58889256 [==============================] - 3s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [7]:
for layer in base_model.layers:
    layer.trainable=False
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [8]:
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.models import Model

last_layer = base_model.get_layer('block5_pool')
last_output = last_layer.output

x = Flatten()(last_output)
x = Dense(64, activation='relu', name='FC2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(2, activation='softmax', name='softmax')(x)

new_model = Model(inputs=base_model.input, outputs=x)
new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [9]:
new_model.compile(Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
new_model.fit_generator(train_batches, steps_per_epoch=4, validation_data=valid_batches, validation_steps=2, epochs=20, verbose=2)

C:\Users\yash\Anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
4/4 - 30s - loss: 1.4141 - accuracy: 0.4750 - val_loss: 1.3304 - val_accuracy: 0.7667
Epoch 2/20
4/4 - 27s - loss: 0.6078 - accuracy: 0.7583 - val_loss: 0.4269 - val_accuracy: 0.9000
Epoch 3/20
4/4 - 28s - loss: 0.2512 - accuracy: 0.8917 - val_loss: 0.2275 - val_accuracy: 0.9333
Epoch 4/20
4/4 - 26s - loss: 0.1693 - accuracy: 0.9107 - val_loss: 0.3084 - val_accuracy: 0.9000
Epoch 5/20
4/4 - 26s - loss: 0.1118 - accuracy: 0.9554 - val_loss: 0.1725 - val_accuracy: 0.9500
Epoch 6/20
4/4 - 26s - loss: 0.0551 - accuracy: 0.9821 - val_loss: 0.1790 - val_accuracy: 0.9333
Epoch 7/20
4/4 - 28s - loss: 0.0540 - accuracy: 0.9821 - val_loss: 0.1878 - val_accuracy: 0.9500
Epoch 8/20
4/4 - 28s - loss: 0.0869 - accuracy: 0.9643 - val_loss: 0.1914 - val_accuracy: 0.9500
Epoch 9/20
4/4 - 29s - loss: 0.0297 - accuracy: 1.0000 - val_loss: 0.2606 - val_accuracy: 0.9000
Epoch 10/20
4/4 - 28s - loss: 0.0301 - accuracy: 0.9911 - val_loss: 0.1730 - val_accuracy: 0.9167
Epoch 11/20
4/4 - 27s - loss:

In [11]:
def load_dataset(path):
    data = load_files(path)
    paths = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']))
    return paths, targets

In [12]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np

test_files, test_targets = load_dataset('data/test')

In [23]:
from keras.preprocessing import image  
from keras.applications.vgg16 import preprocess_input
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

test_tensors = preprocess_input(paths_to_tensor(test_files))


100%|███████████████████████████████████████████████████████████████████████████████| 451/451 [00:01<00:00, 379.25it/s]


In [24]:
acc = new_model.evaluate(test_tensors, test_targets)
print(acc)

15/15 [==============================] - 59s 4s/step - loss: 0.1293 - accuracy: 0.9579
[0.12926456332206726, 0.9578713774681091]
